In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import nengo
from nengo.utils.ensemble import tuning_curves
from PopulationMode import PopulationModeModel

tau_ref = 0.0025 # refactory time
tau_RC = 0.02 # membrane RC time constant
tau_syn = 0.005 # synapse constant
n = 10 # number of neurons
sim_time = 0.001
intercepts = -1 + 2*np.random.rand(1,n)[0]
max_rates = 80 + 50*np.random.rand(1,n)[0]
v0 = 0
dt = 0.0001
rad = 3
seed = None

model = nengo.Network(label="test_populationMode", seed=seed)
with model:
    # 1D Population
    stim = nengo.Node(output=[1])
    pre = nengo.Ensemble(n, dimensions=1, intercepts=intercepts,
            max_rates=max_rates, neuron_type=nengo.LIF(tau_RC, tau_ref, v0),
            label="Pre", radius=rad, seed=seed)
    post = nengo.Ensemble(n, dimensions=1, intercepts=intercepts,
            max_rates=max_rates, neuron_type=nengo.LIF(tau_RC, tau_ref, v0),
            label="Post", radius=rad, seed=seed)
    end = nengo.Node(None, size_in=1)

    # 2D Population
#     stim = nengo.Node(output=[1, 1])
#     pre = nengo.Ensemble(n, dimensions=2, intercepts=intercepts,
#             max_rates=max_rates, neuron_type=nengo.LIF(tau_RC, tau_ref, v0),
#             label="Pre", radius=2)
#     post = nengo.Ensemble(n, dimensions=2, intercepts=intercepts,
#             max_rates=max_rates, neuron_type=nengo.LIF(tau_RC, tau_ref, v0),
#             label="Post", radius=2)
#     end = nengo.Node(None, size_in=2)

        
    # Add connections
    nengo.Connection(stim, pre)
    nengo.Connection(pre, pre) 
    nengo.Connection(post, post)
    conn_stim = nengo.Connection(pre, post)
    conn_end = nengo.Connection(post, end)
    
    origins = [conn_stim.pre]
    stim_probe = nengo.Probe(stim)
    pre_probe = nengo.Probe(pre)
    post_probe = nengo.Probe(post)
    voltage_pre_p = nengo.Probe(pre.neurons, 'voltage')
    voltage_post_p = nengo.Probe(post.neurons, 'voltage')
    end_probe = nengo.Probe(end)

    # Simulate the model for second(s)
    with nengo.Simulator(model, dt=dt) as sim:
        sim.run(sim_time)
        
    # Surrogate population
    eval_points_pre, activities_pre = tuning_curves(pre, sim)
    eval_points_post, activities_post = tuning_curves(post, sim)
    decoders = [sim.data[conn_stim].weights, sim.data[conn_end].weights]
    activities = [activities_pre.T, activities_post.T]
    volt_post = sim.data[voltage_post_p]
    
    params = {'population':post,'origins':origins,'evalpoints':eval_points_post,
              'firingrates':activities,'decoders':decoders,'timerange':sim.trange(),'dt':dt,
              'drive':voltage_pre_p,'voltage_post':voltage_post_p
             }
    surr_pop2D = PopulationModeModel(params)
    surr_pop2D.createNoiseModel()



Building finished in 0:00:01.                                                   
Simulating finished in 0:00:01.                                                 


UnboundLocalError: local variable 'noise_matrix' referenced before assignment

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import nengo
from nengo.utils.ensemble import tuning_curves
from PopulationMode import PopulationModeModel


"""
Test for simulating population mode; simulate both models and compare the 
result by plotting their result

"""
tau_ref = 0.0025 # refactory time
tau_RC = 0.02 # membrane RC time constant
tau_syn = 0.005 # synapse constant
n = 10 # number of neurons
sim_time = 0.001
intercepts = -1 + 2*np.random.rand(1,n)[0]
max_rates = 80 + 50*np.random.rand(1,n)[0]
v0 = 0
dt = 0.0001
rad = 3
seed = None

model = nengo.Network(label="test_populationMode", seed=seed)
with model:
    # 1D Population
    stim = nengo.Node(output=[1])
    pre = nengo.Ensemble(n, dimensions=1, intercepts=intercepts,
            max_rates=max_rates, neuron_type=nengo.LIF(tau_RC, tau_ref, v0),
            label="Pre", radius=rad, seed=seed)
    post = nengo.Ensemble(n, dimensions=1, intercepts=intercepts,
            max_rates=max_rates, neuron_type=nengo.LIF(tau_RC, tau_ref, v0),
            label="Post", radius=rad, seed=seed)
    end = nengo.Node(None, size_in=1)

    # 2D Population
#     stim = nengo.Node(output=[1, 1])
#     pre = nengo.Ensemble(n, dimensions=2, intercepts=intercepts,
#             max_rates=max_rates, neuron_type=nengo.LIF(tau_RC, tau_ref, v0),
#             label="Pre", radius=2)
#     post = nengo.Ensemble(n, dimensions=2, intercepts=intercepts,
#             max_rates=max_rates, neuron_type=nengo.LIF(tau_RC, tau_ref, v0),
#             label="Post", radius=2)
#     end = nengo.Node(None, size_in=2)

        
    # Add connections
    nengo.Connection(stim, pre)
    nengo.Connection(pre, pre) 
    nengo.Connection(post, post)
    conn_stim = nengo.Connection(pre, post)
    conn_end = nengo.Connection(post, end)
    
    origins = [conn_stim.pre]
    stim_probe = nengo.Probe(stim)
    pre_probe = nengo.Probe(pre)
    post_probe = nengo.Probe(post)
    end_probe = nengo.Probe(end)

    # Simulate the model for second(s)
    with nengo.Simulator(model, dt=dt) as sim:
        sim.run(sim_time)
        
    # Surrogate population
    eval_points_pre, activities_pre = tuning_curves(pre, sim)
    eval_points_post, activities_post = tuning_curves(post, sim)
    decoders = [sim.data[conn_stim].weights, sim.data[conn_end].weights]
    activities = [activities_pre.T, activities_post.T]
    
    params = {'population':post,'origins':origins,'evalpoints':eval_points_post,
              'firingrates':activities,'decoders':decoders, 'timerange':sim.trange(), 'dt':dt}
    surr_pop2D = PopulationModeModel(params)
    surr_pop2D.createBiasModel()
    # surr_pop2D.createNoiseModel()
    
    # TODO: The modelled bias does not look right; fix this
    plt.figure()
    plt.plot(eval_points_post, np.transpose(surr_pop2D.bias_values_sim),'k',label='simulated bias')
    plt.plot(eval_points_post, np.transpose(surr_pop2D.bias_values_est),'g',label='modeled bias')
    plt.plot(eval_points_post, np.transpose((surr_pop2D.bias_values_sim-surr_pop2D.bias_values_est)),'b',label='diff')
    plt.legend()
                                